In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import keras 
import csv

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from keras.utils import to_categorical 

#for classifying the images
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization, MaxPool2D,GlobalAveragePooling2D
from keras import applications
from keras.optimizers import SGD, Adam
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.utils import plot_model

#now from main function
from main import *

data = np.load('outputFile.npz', allow_pickle = True)

Using TensorFlow backend.


In [2]:
print(data.files)
    
image_data = data['image_data']
image_names = data['image_names']

['image_data', 'image_names']


In [3]:
x_train, y_train, x_test, y_test, x_val, y_val = split_groups(image_data, image_names)

In [21]:
'''
Alright so I will be trying the googleNet or Inception_module
I am going off the 
https://machinelearningmastery.com/how-to-implement-major-architecture-innovations-for-convolutional-neural-networks/
Tutorial
'''

#This is the very naive version
def naive_inception_module(layer_in, f1, f2, f3):
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
    # 3x3 conv
    conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
    # 5x5 conv
    conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    # concatenate filters, assumes filters/channels last
    layer_out = concatenate([conv1, conv3, conv5, pool], axis = -1)
    return layer_out
 
# define model input
visible = Input(shape=(102,102,3))
# add inception module
layer = naive_inception_module(visible, 64, 128, 32)
# create model
model = Model(inputs=visible, outputs=layer)
# summarize model
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

print(x_train.shape)

history = model.fit(x_train, y_train, batch_size= 64, epochs=20, verbose=1)

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 102, 102, 3)  0                                            
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 102, 102, 64) 256         input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 102, 102, 128 3584        input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 102, 102, 32) 2432        input_18[0][0]                   
___________________________________________________________________________________________

ValueError: Error when checking target: expected concatenate_14 to have 4 dimensions, but got array with shape (7577, 1)